In [1]:
#set up our imports 
import pandas as pd

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson
#load some data into a dataframe
wd = [31, 37, 9, 9, 9, 9, 6, 6, 104]
cols = ['point', 'prov', 'latg', 'latm', 'lngg', 'lngm', 'h', 'number', 'oaci']

dfp = pd.read_fwf('estaciones_smn.txt', widths=wd, names=cols)

dfp = dfp[2:]
dp = {
    'point': str, 'prov': str, 
    'latg': int, 'latm': int, 'lngg': int, 'lngm': int, 
    'h': int, 'number': int,
    'oaci': str
}

for k in dp:
    dfp[k] = dfp[k].astype(dp[k])

dfp['lat'] = dfp['latg'] - (dfp['latm'] / 60)
dfp['lng'] = dfp['lngg'] - (dfp['lngm'] / 60)


v = [8, 6, 6, 101-8-12]
dft = pd.read_fwf("temps.txt", widths=v, names=['date', 'max', 'min', 'loc'])
dft = dft[3:]

dft['ansidate'] = pd.to_datetime(dft['date'], format='%d%m%Y')
del(dft['date'])
dt = {
    'max': float, 'min': float, 
    'ansidate': 'datetime64[ns]',
    'loc': str
}

for k in dt:
    dft[k] = dft[k].astype(dt[k])

dft = dft.iloc[::-1]

#geojson_markers = df_to_geojson(dfp[['point', 'lat', 'lng', 'h']], {'point', 'h'}, lat='lat', lon='lng')
#let's look at the dataframe head
dfp.head()


,point,prov,latg,latm,lngg,lngm,h,number,oaci,lat,lng
2,BASE BELGRANO II,ANTARTIDA,-77,52,-34,34,256,89034,SAYB,-77.866667,-34.566667
3,BASE CARLINI (EX JUBANY),ANTARTIDA,-62,14,-58,38,11,89053,SAYJ,-62.233333,-58.633333
4,BASE ESPERANZA,ANTARTIDA,-63,24,-56,59,24,88963,SAYE,-63.400000,-56.983333
5,BASE MARAMBIO,ANTARTIDA,-64,14,-56,43,198,89055,SAWB,-64.233333,-56.716667
6,BASE ORCADAS,ANTARTIDA,-60,45,-44,43,12,88968,SAYO,-60.750000,-44.716667


In [2]:
dft.head()

,max,min,loc,ansidate
44615,15.1,-1.8,VILLA REYNOLDS AERO,2018-07-15
44614,20.0,9.7,VILLA MARIA DEL RIO SECO,2018-07-15
44613,12.0,2.5,VILLA GESELL AERO,2018-07-15
44612,21.2,4.4,VILLA DOLORES AERO,2018-07-15
44611,9.6,3.1,VIEDMA AERO,2018-07-15


In [25]:
%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from datetime import datetime, timedelta
from ipywidgets import HBox, widgets
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles, Icon,
    LayersControl, Marker, MarkerCluster,
    GeoJSON, FullScreenControl, WidgetControl
)

today = datetime.today().date()
out1 = widgets.Output()
from_date = widgets.DatePicker(description='From', disabled=False)
to_date = widgets.DatePicker(description='To', disabled=False)
dates = HBox([from_date, to_date])

def clear_plt(*args, **kwargs):
    plt.cla()
    plt.clf()
    plt.close()
    out1.clear_output()


m = Map(center=(dfp.lat.mean(), dfp.lng.mean()), zoom=6)
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, (today - timedelta(days=1)).isoformat())

selico = Icon(icon_url='red.svg', icon_size=[38, 95], icon_anchor=[22,94])
ico = Icon(icon_url='blue.svg', icon_size=[38, 95], icon_anchor=[22,94])

markers = []
calculated = []
for i, r in dfp.iterrows():
    mark = Marker(
        location=(r.lat, r.lng),
        draggable=False,
        title=f"{r.point}: {r.h}m",
        alt=r.point,
        icon=ico
    )
    markers.append(mark)

def distance(p1, p2):
    # approximate radius of earth in km
    R = 6373.0
    dlon = abs(p2[0] - p1[0])
    dlat = abs(p2[1] - p1[1])
    a = sin(dlat / 2)**2 + cos(abs(p1[0])) * cos(abs(p2[0])) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

import math

def get_distance(lat_1, lng_1, lat_2, lng_2): 
    d_lat = abs(lat_2) - abs(lat_1)
    d_lng = abs(lng_2) - abs(lng_1)

    temp = (  
         math.sin(d_lat / 2) ** 2 
       + math.cos(abs(lat_1))
       * math.cos(abs(lat_2))
       * math.sin(d_lng / 2) ** 2
    )

    return 6373.0 * (2 * math.atan2(math.sqrt(temp), math.sqrt(1 - temp)))

def place_mark(*args, **kwargs):
    event = kwargs.get('type', None)
    coords = kwargs.get('coordinates', None)
    found = False
    if event == 'preclick':
        diff = .45
        d = dfp[dfp.lat.between(coords[0] - diff, coords[0] + diff) &
                dfp.lng.between(coords[1] - diff, coords[1] + diff)]
        if len(d):
            found = True
            for mark in markers:
                if mark.alt in list(d.point):
                    mark.icon = selico
                else:
                    mark.icon = ico
    if event == 'click' and not found:
        d = {}
        for mark in markers:
            mark.icon = ico
            d[get_distance(*(mark.location), *(coords))] = mark.alt
        lower = {k:d[k] for k in sorted(d.keys())[:3]}
        print(lower)
        x = """mark = Marker(
                location=(r.lat, r.lng),
                draggable=False,
                title=f"{r.point}: {r.h}m",
                alt=r.point,
                icon=ico
            )"""
            

def draw(*args, **kwargs):
    global out1
    for m in markers:
        if m.icon is selico:
            data = dft[dft["loc"].str.startswith(m.alt)]
            frmd = from_date.value or datetime(2010, 1, 1)
            tod = to_date.value or today
            data = data[data.ansidate.between(frmd, tod)]
            ax = data.plot(x='ansidate', y=['min', 'max'])
            ax.set(xlabel="Date", ylabel="Temp")
            plt.xticks(rotation=90)
            with out1:
                plt.title(f"{m.title}")
                plt.show()


btn1 = widgets.Button(description="Draw")
btn1.on_click(draw)
btn2 = widgets.Button(description="Clear")
btn2.on_click(clear_plt)
btn = HBox([btn1, btn2])


m.add_layer(nasa_layer)
m.add_layer(MarkerCluster(markers=markers, name="SMN"))

m.on_interaction(place_mark)

m.add_control(WidgetControl(widget=dates, position="bottomleft"))
m.add_control(WidgetControl(widget=btn, position='bottomright'))
m.add_control(WidgetControl(widget=out1, position='bottomright'))
m.add_control(FullScreenControl())
m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

{30.742592738351178: 'GENERAL PICO AERO', 729.5054059206556: 'BASE ORCADAS', 1290.550554879065: 'BASE SAN MARTIN'}
{356.9396825109895: 'RECONQUISTA AERO', 625.7109252831882: 'RIVADAVIA', 1609.2850055349963: 'SANTA TERESITA AERO'}
{1275.8339599555031: 'RIO COLORADO', 1439.160248738581: 'SAN LUIS AERO', 1700.010946289227: 'COMODORO RIVADAVIA AERO'}
{772.6694969436238: 'AEROPARQUE AERO', 1119.9099802730886: 'BUENOS AIRES', 1437.3650246209143: 'SAN FERNANDO'}
{772.6694969436238: 'AEROPARQUE AERO', 1119.9099802730886: 'BUENOS AIRES', 1437.3650246209143: 'SAN FERNANDO'}
{361.33145404884306: 'DOLORES AERO', 1070.9271875503612: 'MONTE CASEROS AERO', 1073.8165443520975: 'GOBERNADOR GREGORES AERO'}
{656.4017079893825: 'GOBERNADOR GREGORES AERO', 755.8764555452991: 'VILLA MARIA DEL RIO SECO', 868.7482903500065: 'DOLORES AERO'}
